In [1]:
import pandas as pd
from datetime import datetime
import numpy as np

We import our dataset

In [2]:
data_path="data/fires-all.csv"
try:
    fires=pd.read_csv(data_path)
except Exception as error:
    print(f"Error while importing the excel file: {error}")
fires.head()

In [3]:
fires.info()

### Remove unnecesary columns

### Remove unnecesary rows

In [4]:
#We only want the fires of the comunidad 3 (Galicia)
fires=fires[fires["idcomunidad"]==3]
#We drop null values on lat and lng, beacsue we cannot find where the fire happend,
#  and it is older data from  1968
fires=fires.dropna(subset=["lat","lng"])
#We remove the column idcomunidad as it is no longer need it
fires.drop(["idcomunidad"],axis=1,inplace=True)
#print how many null values are for each column
fires.isna().sum(),
#TODO: from the coordinates lat and lng obtain the blanks municipio

In [5]:
#We save the data for the time series prediction
data_path="data/fires-time-series.xlsx"
try:
    fires.to_excel(data_path,index=False)
except Exception as error:
    print(f"Error while exporting the data to the excel file: {error}")

In [6]:
data_path="data/fires-time-series.xlsx"
try:
    fires_time_series=pd.read_excel(data_path)
except Exception as error:
    print(f"Error while importing the excel file: {error}")
fires.head()

#### Filtramos las fechas que sean mas antiguas del 2005

#### Juntamos todos los datos en un unico csv por estacion meteo


In [7]:

year_data = pd.read_csv("data/2008_2009/C02_Boimorto_01_01_2008_01_01_2009.csv",  encoding='utf-8', delimiter=',', decimal='.')
year_data.head()


In [8]:
#Definimos las estaciones de meteorologia
# stations=["C01_A Capela_","C02_Boimorto_","LU01_Castro de Rei_","LU02_Monforte de Lemos_"]
stations=["C01_A Capela_"]
station_data = {station: pd.DataFrame() for station in stations}

dataset_all = pd.DataFrame()

#Iteramos por todos los años que tenemos disponibles de datos meteorologicos
for year in range(2005,2019):
    #Actualizamos el data path
    data_path=f"data/{year}_{year+1}/"
    #Por cada estacion lo abrimos y lo juntamos
    for station in stations:
        try:
            station_path=data_path+f"{station}01_01_{year}_01_01_{year+1}.csv"
            
            aux = pd.read_csv(station_path)

            dataset_all = pd.concat([dataset_all, aux], ignore_index=True)

            #Guardamos en station data
            station_data[station].concat([station_data[station],aux],ignore_index=True)
            
        except pd.errors.ParserError as parse_error:
            print(f"Parser error when reading {station_path}: {parse_error}")
        except FileNotFoundError as file_error:
            print(f"File not found: {station_path}")
        except Exception as general_error:
            print(f"General error occurred while reading the file: {general_error}")
    print(f"Datos cargados para del año {year}")


dataset_all.show()
station_data["C01_A Capela_"].show()

In [ ]:
fires.isna().sum()

In [ ]:
fires.info()

In [ ]:
#This si done to avoid repeating teh fetching of weather that is an intensive task
data_path="data/fires-weather.xlsx"
try:
    fires=pd.read_excel(data_path)
except Exception as error:
    print(f"Error while importing the excel file: {error}")
fires.head()

### Group by weeks

### Split train and test data

In [ ]:
#We split 80% of the data for training the model
train_split=80


## Data cleaning

### Set up

We first import the different libraries that we will be using for this project

We import our dataset

In [3]:
fires.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284589 entries, 0 to 284588
Data columns (total 21 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id               284589 non-null  int64  
 1   superficie       284589 non-null  float64
 2   fecha            284589 non-null  object 
 3   lat              230154 non-null  float64
 4   lng              230154 non-null  float64
 5   latlng_explicit  284589 non-null  int64  
 6   idcomunidad      284589 non-null  int64  
 7   idprovincia      284589 non-null  int64  
 8   idmunicipio      284589 non-null  int64  
 9   municipio        284576 non-null  object 
 10  causa            284589 non-null  int64  
 11  causa_supuesta   284589 non-null  int64  
 12  causa_desc       284589 non-null  int64  
 13  muertos          284589 non-null  int64  
 14  heridos          284589 non-null  int64  
 15  time_ctrl        284589 non-null  int64  
 16  time_ext         284589 non-null  int6

### Remove unnecesary rows

In [5]:
#We save the data for the time series prediction
data_path="data/fires-time-series.xlsx"
try:
    fires.to_excel(data_path,index=False)
except Exception as error:
    print(f"Error while exporting the data to the excel file: {error}")

Error while exporting the data to the excel file: No module named 'openpyxl'


#### Filtramos las fechas que sean mas antiguas del 2005

In [7]:

year_data = pd.read_csv("data/2008_2009/C02_Boimorto_01_01_2008_01_01_2009.csv",  encoding='utf-8', delimiter=',', decimal='.')
year_data.head()


,IdProvincia,IdEstacion,Fecha,Año,Dia,Temp Media (ºC),Temp Max (ºC),Hora Temp Max,Temp Mínima (ºC),Hora Temp Min,...,DirViento (º),VelVientoMax (m/s),Hora VelMax,Dir viento Vel Max (º),Radiación (MJ/m2),Precipitación (mm),PePMon,EtPMon,Temp Suelo 10 cm (ºC),Temp Suelo 30 cm (ºC)
0,15,2,01/01/2008,2008,1,7.12,10.22,14:10,3.86,02:30,...,152.3,10.59,22:37,151.7,3.37,9.8,5.47,0.93,NaN,NaN
1,15,2,02/01/2008,2008,2,7.86,10.02,15:40,6.24,07:30,...,181.1,9.01,23:02,153.0,3.01,26.6,16.33,0.58,NaN,NaN
2,15,2,03/01/2008,2008,3,5.97,8.44,01:20,4.31,18:50,...,198.4,8.17,01:32,229.9,2.54,17.8,10.87,0.48,NaN,NaN
3,15,2,04/01/2008,2008,4,6.26,9.48,23:10,2.07,07:40,...,186.2,6.31,22:51,210.6,1.98,0.0,0.00,0.43,NaN,NaN
4,15,2,05/01/2008,2008,5,11.69,13.46,13:50,9.22,00:10,...,228.3,7.78,16:05,275.9,0.83,2.2,0.00,0.37,NaN,NaN


In [ ]:
fires.isna().sum()

In [ ]:
#This si done to avoid repeating teh fetching of weather that is an intensive task
data_path="data/fires-weather.xlsx"
try:
    fires=pd.read_excel(data_path)
except Exception as error:
    print(f"Error while importing the excel file: {error}")
fires.head()

### Split train and test data

### Analyze train data set

In [ ]:
#TODO

In [ ]:
#TODO

In [ ]:
#TODO

## Test Model
